In [2]:
#필요한 라이브러리 설치
import csv
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db
from haversine import haversine
import sqlite3

In [3]:
#csv데이터를 2차원 리스트로 불러오기
f = open('cultureFestival.csv', 'r')
reader = csv.reader(f)
rows = []
for i in reader:
        rows.append(i) #파일 가져오기
f.close()

#default firebase database 설정
cred = credentials.Certificate('dbKey.json')
firebase_admin.initialize_app(cred, {
    'databaseURL' : 'https://dbproject-b1bec-default-rtdb.firebaseio.com/' 
    
})

In [7]:
#csv데이터를 2차원 리스트로 불러오기
f = open('cultureFestival.csv', 'r')
reader = csv.reader(f)
rows = []
for i in reader:
        rows.append(i) #파일 가져오기
f.close()


#rows를 sqlite에 저장한다.
conn = sqlite3.connect('cultureFstv.db')
cursor = conn.cursor()
cursor.execute('DROP TABLE mytable') #db 초기화
cursor.execute('''
    CREATE TABLE IF NOT EXISTS mytable (
        festvlNm TEXT,
        opar TEXT,
        fsvlStartData Text,
        fstvEndData Text,
        fstvlCo Text,
        mnnstNm Text,
        auspcInsttNm Text,
        suprtInsttNm Text,
        phoneNumber Text,
        humpageUrl Text,
        realteInfo Text,
        rdnmadr Text,
        lnmadr Text,
        latitude Text,
        longitude Text,
        referenceData Text,
        instt_code Text,
        instt_nm Text,
        distance Text
    )
''')

for i in range(1, len(rows)):
    cursor.execute('INSERT INTO mytable VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', rows[i])
    conn.commit()

keyword = "인천" #안드로이드에서 사용자에게 입력받은 값

# 파이어베이스에 데이터 업로드
db_ref = db.reference('search')
db_ref.delete() #디렉토리 초기화
results = cursor.execute(f"SELECT * FROM mytable WHERE festvlNm LIKE '%{keyword}%' LIMIT 10").fetchall()

cnt = 0
for i in results:
    data = {
            '축제 이름': i[0],
            '개최지': i[1],
            '시작일': i[2],
            '종료일': i[3],
            '축제 내용': i[4],
            '전화번호': i[5],
            '홈페이지 주소': i[6],
            '관련 정보': i[7],
            '도로명 주소': i[8],
            '지번 주소': i[9],
            '거리': i[10]
        }
    db_ref.child(str(cnt)).set(data)
    cnt += 1

